In [13]:
import sys
import argparse
from mlfs.airquality import util

parser = argparse.ArgumentParser()

parser.add_argument("--country", default="spain")
parser.add_argument("--city", default="malaga")
parser.add_argument("--street", default="el-atabal")
parser.add_argument("--sensor-url", default="https://api.waqi.info/feed/@8487")
parser.add_argument("--csv-filename", default="air-quality-data.csv")
parser.add_argument("--latitude", default="TODO")
parser.add_argument("--longitude", default="TODO")

args, unknown = parser.parse_known_args()

aqicn_url = args.sensor_url
country = args.country
city = args.city
street = args.street
csv_filename = args.csv_filename

sensor_suffix = util.construct_sensor_suffix(country, city, street)
weather_suffix = util.construct_weather_suffix(country, city)

print(sensor_suffix)

# print out the arg values
print(f"Provided args: \nUrl: {aqicn_url}\nCountry: {country}\nCity: {city}\nStreet: {street}\nFilename: {csv_filename}")

spain_malaga_carril_del_capitan
Provided args: 
Url: https://api.waqi.info/feed/A203590/
Country: spain
City: malaga
Street: carril_del_capitan
Filename: carril_del_capitan.csv


In [15]:
import sys
from pathlib import Path
import os

def is_google_colab() -> bool:
    if "google.colab" in str(get_ipython()):
        return True
    return False

def clone_repository() -> None:
    !git clone https://github.com/featurestorebook/mlfs-book.git
    %cd mlfs-book

def install_dependencies() -> None:
    !pip install --upgrade uv
    !uv pip install --all-extras --system --requirement pyproject.toml

if is_google_colab():
    clone_repository()
    install_dependencies()
    root_dir = str(Path().absolute())
    print("Google Colab environment")
else:
    root_dir = Path().absolute()
    # Strip ~/notebooks/ccfraud from PYTHON_PATH if notebook started in one of these subdirectories
    if root_dir.parts[-1:] == ('airquality',):
        root_dir = Path(*root_dir.parts[:-1])
    if root_dir.parts[-1:] == ('notebooks',):
        root_dir = Path(*root_dir.parts[:-1])
    root_dir = str(root_dir)
    print("Local environment")

# Add the root directory to the `PYTHONPATH` to use the `recsys` Python module from the notebook.
if root_dir not in sys.path:
    sys.path.append(root_dir)
print(f"Added the following directory to the PYTHONPATH: {root_dir}")

# Set the environment variables from the file <root_dir>/.env
from mlfs import config
if os.path.exists(f"{root_dir}/.env"):
    settings = config.HopsworksSettings(_env_file=f"{root_dir}/.env")

Local environment
Added the following directory to the PYTHONPATH: /home/pshvaiko/Desktop/kth_scalable_ml/hw1/mlfs-book
HopsworksSettings initialized!


<span style="font-width:bold; font-size: 3rem; color:#333;">- Part 02: Daily Feature Pipeline for Air Quality (aqicn.org) and weather (openmeteo)</span>

## 🗒️ This notebook is divided into the following sections:
1. Download and Parse Data
2. Feature Group Insertion


__This notebook should be scheduled to run daily__

In the book, we use a GitHub Action stored here:
[.github/workflows/air-quality-daily.yml](https://github.com/featurestorebook/mlfs-book/blob/main/.github/workflows/air-quality-daily.yml)

However, you are free to use any Python Orchestration tool to schedule this program to run daily.

### <span style='color:#ff5f27'> 📝 Imports

In [16]:
import datetime
import time
import requests
import pandas as pd
import hopsworks
from mlfs.airquality import util
from mlfs import config
import json
import warnings
warnings.filterwarnings("ignore")

## <span style='color:#ff5f27'> 🌍 Get the Sensor URL, Country, City, Street names from Hopsworks </span>

__Update the values in the cell below.__

__These should be the same values as in notebook 1 - the feature backfill notebook__


In [17]:
project = hopsworks.login()
fs = project.get_feature_store() 
secrets = hopsworks.get_secrets_api()

# This line will fail if you have not registered the AQICN_API_KEY as a secret in Hopsworks
AQICN_API_KEY = secrets.get_secret("AQICN_API_KEY").value
all_locations_str = secrets.get_secret("SENSOR_LOCATION_JSON").value
all_locations = json.loads(all_locations_str)

print(all_locations)

location_id = util.construct_sensor_id(country, city, street)
location = all_locations[location_id]

country=location['country']
city=location['city']
street=location['street']
aqicn_url=location['aqicn_url']
latitude=location['latitude']
longitude=location['longitude']

today = datetime.date.today()

location

2025-11-18 10:50:27,475 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-11-18 10:50:27,478 INFO: Initializing external client
2025-11-18 10:50:27,479 INFO: Base URL: https://c.app.hopsworks.ai:443


2025-11-18 10:50:29,156 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1298580
{'country': 'spain', 'city': 'malaga', 'street': 'el-atabal', 'aqicn_url': 'https://api.waqi.info/feed/@8487', 'latitude': 36.72, 'longitude': -4.42, 'spain/malaga/el-atabal': {'country': 'spain', 'city': 'malaga', 'street': 'el-atabal', 'aqicn_url': 'https://api.waqi.info/feed/@8487', 'latitude': 36.72, 'longitude': -4.42}, 'spain/malaga/carril_del_capitan': {'country': 'spain', 'city': 'malaga', 'street': 'carril_del_capitan', 'aqicn_url': 'https://api.waqi.info/feed/A203590/', 'latitude': 36.72, 'longitude': -4.42}}


{'country': 'spain',
 'city': 'malaga',
 'street': 'carril_del_capitan',
 'aqicn_url': 'https://api.waqi.info/feed/A203590/',
 'latitude': 36.72,
 'longitude': -4.42}

### <span style="color:#ff5f27;"> 🔮 Get references to the Feature Groups </span>

In [18]:
# Retrieve feature groups
air_quality_fg = fs.get_feature_group(
    name=f'air_quality_lagged_{sensor_suffix}',
    version=1,
)
weather_fg = fs.get_feature_group(
    name=f'weather_{weather_suffix}',
    version=1,
)

---

## <span style='color:#ff5f27'> 🌫 Retrieve Today's Air Quality data (PM2.5) from the AQI API</span>


In [19]:
import requests
import pandas as pd

aq_today_df = util.get_pm25(aqicn_url, country, city, street, today, AQICN_API_KEY)
aq_today_df

,pm25,country,city,street,date,url
0,1.0,spain,malaga,carril_del_capitan,2025-11-18,https://api.waqi.info/feed/A203590/


In [20]:
aq_today_df.info()

history_df = air_quality_fg.read()
sensor_history = history_df[
    (history_df['country'] == country) &
    (history_df['city'] == city) &
    (history_df['street'] == street)
].sort_values('date')

# 2. Normalize date to be tz-naive (no timezone) in BOTH frames
sensor_history['date'] = pd.to_datetime(sensor_history['date']).dt.tz_localize(None)
aq_today_df['date'] = pd.to_datetime(aq_today_df['date']).dt.tz_localize(None)

# 2. Append today's row (no lags yet) to the history
combined = pd.concat([sensor_history, aq_today_df], ignore_index=True)
combined = combined.sort_values('date')

# 3. Recompute lagged features for all rows
for lag in [1, 2, 3]:
    combined[f'pm25_lag_{lag}'] = combined['pm25'].shift(lag)

# 4. Take the last row (today) with lagged features
aq_today_with_lags = combined.tail(1)
aq_today_with_lags



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1 entries, 0 to 0
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype         
---  ------   --------------  -----         
 0   pm25     1 non-null      float32       
 1   country  1 non-null      object        
 2   city     1 non-null      object        
 3   street   1 non-null      object        
 4   date     1 non-null      datetime64[ns]
 5   url      1 non-null      object        
dtypes: datetime64[ns](1), float32(1), object(4)
memory usage: 176.0+ bytes
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (1.05s) 


,date,pm25,country,city,street,url,pm25_lag_1,pm25_lag_2,pm25_lag_3
1350,2025-11-18,1.0,spain,malaga,carril_del_capitan,https://api.waqi.info/feed/A203590/,0.1,0.1,0.15


In [21]:
air_quality_fg.insert(aq_today_with_lags, wait=True)

2025-11-18 10:51:28,247 INFO: 	1 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1298580/fs/1286211/fg/1721937


Uploading Dataframe: 100.00% |██████████| Rows 1/1 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: air_quality_lagged_spain_malaga_carril_del_capitan_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1298580/jobs/named/air_quality_lagged_spain_malaga_carril_del_capitan_1_offline_fg_materialization/executions
2025-11-18 10:52:15,759 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-18 10:52:18,928 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 10:52:22,102 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 10:54:04,418 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 10:54:04,608 INFO: Waiting for log aggregation to finish.
2025-11-18 10:54:30,085 INFO: Execution finished successfully.


(Job('air_quality_lagged_spain_malaga_carril_del_capitan_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "pm25",
           "min_value": -0.1,
           "max_value": 500.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762122
         }
       },
       "result": {
         "observed_value": 1.0,
         "element_count": 1,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T09:51:28.000246Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     }
   ],
   "evaluation_parameters": {},
   "statistics": {
     "evaluated_expec

## <span style='color:#ff5f27'> 🌦 Get Weather Forecast data</span>

In [22]:
hourly_df = util.get_hourly_weather_forecast(city, latitude, longitude)
hourly_df = hourly_df.set_index('date')

# We will only make 1 daily prediction, so we will replace the hourly forecasts with a single daily forecast
# We only want the daily weather data, so only get weather at 12:00
daily_df = hourly_df.between_time('11:59', '12:01')
daily_df = daily_df.reset_index()
daily_df['date'] = pd.to_datetime(daily_df['date']).dt.date
daily_df['date'] = pd.to_datetime(daily_df['date'])
daily_df['city'] = city
daily_df

Coordinates 36.5°N -4.75°E
Elevation 20.0 m asl
Timezone None None
Timezone difference to GMT+0 0 s


,date,temperature_2m_mean,precipitation_sum,wind_speed_10m_max,wind_direction_10m_dominant,city
0,2025-11-18,17.500000,0.0,5.001280,239.743652,malaga
1,2025-11-19,17.500000,0.0,5.623380,219.805527,malaga
2,2025-11-20,17.549999,0.0,6.608722,240.642334,malaga
3,2025-11-21,14.300000,0.0,26.280001,360.000000,malaga
4,2025-11-22,16.250000,0.0,7.636753,44.999897,malaga
5,2025-11-23,16.549999,0.0,9.422101,313.451904,malaga
6,2025-11-24,17.799999,0.0,13.684735,243.435013,malaga


In [23]:
daily_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7 entries, 0 to 6
Data columns (total 6 columns):
 #   Column                       Non-Null Count  Dtype         
---  ------                       --------------  -----         
 0   date                         7 non-null      datetime64[ns]
 1   temperature_2m_mean          7 non-null      float32       
 2   precipitation_sum            7 non-null      float32       
 3   wind_speed_10m_max           7 non-null      float32       
 4   wind_direction_10m_dominant  7 non-null      float32       
 5   city                         7 non-null      object        
dtypes: datetime64[ns](1), float32(4), object(1)
memory usage: 356.0+ bytes


## <span style="color:#ff5f27;">⬆️ Uploading new data to the Feature Store</span>

In [11]:
# Insert new data
# air_quality_fg.insert(aq_today_df)

In [24]:
# Insert new data
weather_fg.insert(daily_df, wait=True)

2025-11-18 10:54:38,719 INFO: 	2 expectation(s) included in expectation_suite.
Validation succeeded.
Validation Report saved successfully, explore a summary at https://c.app.hopsworks.ai:443/p/1298580/fs/1286211/fg/1721934


Uploading Dataframe: 100.00% |██████████| Rows 7/7 | Elapsed Time: 00:01 | Remaining Time: 00:00


Launching job: weather_spain_malaga_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai:443/p/1298580/jobs/named/weather_spain_malaga_1_offline_fg_materialization/executions
2025-11-18 10:55:14,451 INFO: Waiting for execution to finish. Current state: INITIALIZING. Final status: UNDEFINED
2025-11-18 10:55:17,719 INFO: Waiting for execution to finish. Current state: SUBMITTED. Final status: UNDEFINED
2025-11-18 10:55:20,906 INFO: Waiting for execution to finish. Current state: RUNNING. Final status: UNDEFINED
2025-11-18 10:57:12,922 INFO: Waiting for execution to finish. Current state: AGGREGATING_LOGS. Final status: SUCCEEDED
2025-11-18 10:57:13,066 INFO: Waiting for log aggregation to finish.
2025-11-18 10:57:38,505 INFO: Execution finished successfully.


(Job('weather_spain_malaga_1_offline_fg_materialization', 'SPARK'),
 {
   "success": true,
   "results": [
     {
       "success": true,
       "expectation_config": {
         "expectation_type": "expect_column_min_to_be_between",
         "kwargs": {
           "column": "wind_speed_10m_max",
           "min_value": -0.1,
           "max_value": 1000.0,
           "strict_min": true
         },
         "meta": {
           "expectationId": 762118
         }
       },
       "result": {
         "observed_value": 5.001279830932617,
         "element_count": 7,
         "missing_count": null,
         "missing_percent": null
       },
       "meta": {
         "ingestionResult": "INGESTED",
         "validationTime": "2025-11-18T09:54:38.000718Z"
       },
       "exception_info": {
         "raised_exception": false,
         "exception_message": null,
         "exception_traceback": null
       }
     },
     {
       "success": true,
       "expectation_config": {
         "expect

## <span style="color:#ff5f27;">⏭️ **Next:** Part 03: Training Pipeline
 </span> 

In the following notebook you will read from a feature group and create training dataset within the feature store
